In [56]:
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math as mt
from math import log
import itertools
import random

In [57]:
data_path = ''

In [58]:
df_edges_1 = pd.read_csv(os.path.join(data_path,'2001_2002_edges.csv'))

In [59]:
df_filtered_authorships_1 = pd.read_csv(os.path.join(data_path,'2001_2002_filtered_authorships.csv'))

In [61]:
df_edges_2 = pd.read_csv(os.path.join(data_path,'2003_2004_edges.csv'))

In [67]:
df_filtered_authorships_2 = pd.read_csv(os.path.join(data_path,'2003_2004_filtered_authorships.csv'))

In [63]:
df_nodes_catalogue = pd.read_csv(os.path.join(data_path,'nodes_catalogue.csv'))

In [64]:
df_nodes_catalogue.shape

(93912, 2)

In [65]:
G = nx.Graph()


In [66]:
G.add_nodes_from(df_nodes_catalogue.author.values)

In [69]:
df_filtered_authorships = pd.concat([df_filtered_authorships_1, df_filtered_authorships_2])

In [70]:
df_edges = pd.concat([df_edges_1, df_edges_2])

In [71]:
G.add_edges_from(df_edges.values)

In [10]:
#G.nodes

In [11]:
#G.edges

In [72]:
def nodes_connected(u, v):
    return u in G.neighbors(v)

In [74]:
N = 40000

In [75]:
tasa_positivos = 1/2

In [76]:
positivos = round(N*(tasa_positivos))
negativos = round(N*(1-tasa_positivos))

In [77]:
positivos,negativos

(20000, 20000)

In [78]:
lista_no_conectados= []
no_conectados = 0
while no_conectados<negativos:
    a = b = ''
    while a == b:
        tuple_ = df_nodes_catalogue.author.sample(2).values
        a = tuple_[0]
        b = tuple_[1]

    #están conectados?
    con = nodes_connected(a, b)
    if con==False:
        lista_no_conectados.append([a,b])
        no_conectados+=1
    

In [79]:
no_conectados = pd.DataFrame(lista_no_conectados).rename(columns={0:'source',1:'target'})

In [80]:
conectados = df_edges.sample(positivos)

In [81]:
conectados['connected'] = 1

In [82]:
X = conectados.append(no_conectados).fillna(0)

In [83]:
X.connected.value_counts()

0.0    20000
1.0    20000
Name: connected, dtype: int64

In [84]:
X.drop_duplicates()

,source,target,connected
6497,Hideaki Takanobu,Kayoko Ohtsuki,1.0
54953,Joseph Douglas Horton,Alejandro López-Ortiz,1.0
72287,Siddhartha R. Dalal,Ashish Jain,1.0
41022,Laurence Melloul,Armando Fox,1.0
41160,Riccardo Bettati,Wei Zhao 0001,1.0
...,...,...,...
19995,Florian Schiel,Walt Truszkowski,0.0
19996,Goffredo Haus,Mizuho Iwaihara,0.0
19997,Joe L. Armstrong,George T. Duncan,0.0
19998,Wayne C. Boncyk,Mike Hinchey,0.0


In [85]:
X.to_pickle('sample_wo_features_2.links_prediction')

In [86]:
sample = X.drop_duplicates().reset_index(drop=True).copy()

In [87]:
sample.connected.value_counts(1)

0.0    0.524893
1.0    0.475107
Name: connected, dtype: float64

# Pipeline

In [88]:
dict_count_papers = df_nodes_catalogue.set_index('author').to_dict()['id_article']

## Aggregated Features

### Sum of papers

In [89]:
sample['sum_of_papers'] = sample[['source','target']].apply(lambda x: dict_count_papers[x[0]]+dict_count_papers[x[1]], axis=1)

In [90]:
sample.head()

,source,target,connected,sum_of_papers
0,Hideaki Takanobu,Kayoko Ohtsuki,1.0,17
1,Joseph Douglas Horton,Alejandro López-Ortiz,1.0,26
2,Siddhartha R. Dalal,Ashish Jain,1.0,19
3,Laurence Melloul,Armando Fox,1.0,21
4,Riccardo Bettati,Wei Zhao 0001,1.0,76


### Sum of neighbors

In [91]:
sample['sum_of_neighbors'] =sample[['source','target']].apply(lambda x: len([x for x in G.neighbors(x[0])]) + len([x for x in G.neighbors(x[1])]), axis=1)

In [92]:
#sample['sum_of_neighbors'].value_counts()

In [93]:
#sample[sample['sum_of_neighbors']==7]

### Sum of log (Secundary Neighbors count)

In [94]:
#l = [G.neighbors(y) for y in [x for x in G.neighbors('Alberto L. Sangiovanni-Vincentelli')]]

In [95]:
#[x for x in G.neighbors('Alberto L. Sangiovanni-Vincentelli')]

In [96]:
def extract_sec_neig(node):
    try:
        l = [G.neighbors(y) for y in [x for x in G.neighbors(node)]]
        secondary_neighbors = []
        for x in l:
            secondary_neighbors.extend([z for z in x])
        if 0==len(secondary_neighbors):
            return 1

        return len(secondary_neighbors)
        #return len(set(secondary_neighbors))
    except Exception as e:
        print(node,e)

In [97]:
def sum_sec_neig(node_a,node_b):
    try:
        return log(extract_sec_neig(node_a))+log(extract_sec_neig(node_b))
    except Exception as e:
        print(e)
        print(node_a,'_',node_b)
    

In [98]:
sample['log_secundary_neighbors']= sample[['source','target']].apply(lambda x: sum_sec_neig(x[0],x[1]), axis=1,)

### Shortest Distance

In [99]:
def extract_lenght_short_path(list_nodes):
    node_a,node_b = list_nodes[0],list_nodes[1]
    try:
        return nx.single_source_shortest_path_length(G,node_a)[node_b]
    except Exception as e:
        print(e)
        return np.nan        

In [100]:
from multiprocessing import Pool
p = Pool()

'Terry S. Griggs'
'Constance Fleuriot'
'Thien T. Bui'
'Peter Wendt'
'Tien Chi Chen'
'Jean-Claude Bermond'
'Henry Markram'
'Yariv Aridor'
'Walter F. L. Bogaerts'
'J. C. Mignot'
'Liming Chen 0002'
'Y. T. Wu'
'James Bohn'
'Toshio Tokita'
'Ai Oshiro'
'Alejandra Lugo'
'Peter Thorne'
'Deborah Goldman'
'L. Zhang'
'John W. Dalle Molle'
'Robert Patton Leland'
'Man Hon Wong 0001'
'Doug Fang'
'Yuzuru Matsuoka'
'Wendy G. Lehnert'
'Takakazu Mori'
'Chi-Kwong Li'
'J. R. Parker'
'Keun-Hyung Kim'
'Mehrad Yasrebi'
'David Bedford'
'Phuoc Tran-Gia'
'Byeong-Seok Shin'
'Stephanie L. Reel'
'Terry M. Stepien'
'Heinz Dibold'
'Wolfgang Raab'
'A. B. Dobrzeniecki'
'Donald J. McCubbrey'
'Hormoz Djahanshahi'
'Cary B. Phillips'
'Josef Plankl'
'Muhammad Sarfraz'
'Eric Nöcker'
'Vincenzo Catania'
'Andreas Kipp 0001'
'Robert R. Korfhage'
'Mark Gerhardt'
'Jesus A. Gonzalez'
'Mark Short'
'Ziauddin Ahmad'
'Colleen Bushell'
'Peter Seipel'
'Chorkin Chan'
'Ray Bird'
'María C. Fernández-Baizán'
'William M. Shaw Jr.'
'Fumio Kan

'Otto E. Laske'
'Jong-Yih Kuo'
'Andreas Blaszczyk'
'Toru Takaki'
'Pascal Fua'
'Seung H. Hwang'
'Fleur L. Steven'
'Vicente López 0002'
'Benjamin F. Hobbs'
'F. J. Smith'
'Hirofumi Sanada'
'Manoj K. Chari'
'Hyogon Kim'
'Robert John Walters'
'Francisco J. Ballesteros'
'Simon Emmerson'
'Daniel R. Jeske'
'Alexander M. Haimovich'
'Roy Dz-Ching Ju'
'Lalit M. Patnaik'
'R. Rajagopalan'
'Yung Taek Kim'
'Zheng Min'
'Robert H. Baud'
'Brian A. Nejmeh'
'José Carlos Segura Luna'
'Shih-Hsu Huang'
'Emma J. McCoy'
'Isao Arima'
'Damien Trentesaux'
'Kazuhiro Hayashi'
'Graham J. Reynolds'
'U. Schnell'
'Zdzislaw Czarnul'
'Daniel Hunter'
'Antoine Danchin'
'John A. Reed'
'Gianfranco Doretto'
'Bernhard Lauth'
'Milan Ceska 0001'
'Mahmoud El-Alem'
'Dieter Nazareth'
'Sashidhar P. Reddi'
'I. S. W. B. Prasetya'
'Lisa R. Schiff'
'Akihiro Kanamori'
'Khee Meng Koh'
'Giuseppe Megali'
'Carol A. Bean'
'Joachim Wehler'
'David A. Southard'
'Christian Zanardi'
'Szu-Tsung Cheng'
'Avishai Wool'
'Lakshman Prasad'
'Ronald B. Ste

'Gregory J. McRae'
'Brenton D. Gray'
'Stefano Messelodi'
'Bob Goedhart'
'John Schwegler'
'Ilya B. Muchnik''Tomas A. Lipinski'

'Shin Nakajima 0002'
'Pierre Duchet'
'Xinyou Li'
'Thomas Hain'
'Didier Juvin'
'Steve Counsell'
'Steven P. Ketchpel'
'Paolo Prandoni'
'Bernard Virot''Jiandong Huang'

'Horace J. Meng'
'Takashi Ohtani'
'Yaakov Kogan'
'Dino Oliva'
'Lionel Revéret'
'Wael A. Hashlamoun'
'Hang Li 0001'
'Abdellah El Moudni'
'Joel D. Martin'
'Douglas N. Zuckerman'
'Brigitte Zellner'
'Alexandru Cicortas'
'Shipeng Li'
'Hans-Rüdiger Pfister'
'Peter V. Coveney'
'Susan H. Xu'
'Chwan-Hwa John Wu'
'Roger Rodriguez'
'William J. Premerlani'
'P. C. Ojha'
'Jacques Nicolas'
'Gheith A. Abandah'
'Ahmed Guessoum'
'Amit Das'
'Gary S. Tyson'
'Tomasz Janowski'
'Andrew Monk'
'Mario Paludetto'
'Daniel V. Klein''Kyeong-Man Kim'

'Hager Kammoun'
'Takehito Tsukada'
'Zarine Kemp'
'Scott E. Budge'
'Bedrich J. Hosticka'
'Seth D. Silverstein'
'Frederic Le Mouel'
'Ken Tang'
'L. H. Yeo'
'Christine W. Chan'
'Mark B

'Gísli Hjálmtýsson''Steve Winker'

'Sophie Cherki''David Chen 0001''Masoud Mohammadian'

'Murray Cole'
'Kuniaki Nagayama'
'Christian Blum 0001'

'Don Davis''Ralf Knirsch'

'Wencheng Wu'
'Bilel Gargouri''Klaus Hofmann'
'Przemyslaw Rokita'
'Jianwen Su'
'Nader Mohsenian'

'Ansgar Bernardi''Fuxing Li'
'Carol L. Walker'

'Muhammad Kafil''Hadar Shemtov'

'Edson Emílio Scalabrin'
'Ryoji Suzuki'
'Robert Lewis''Siming Lin'

'S. Ragupathi''K. T. Sun'

'Vyacheslav Petukhin'
'Lynn Elliott'
'Kapsu Kim''Pedro M. Crespo'

'Rémi Ronfard'
'Stephen Suen'
'Ambar Chowdhury'
'Seong-Ok Bae''Gerald F. Smith'

'Martin Große-Rhode''Hugues Fauconnier'

'Albert Koelmans'
'Chien-Le Goh'
'Mark A. Turnquist'
'Tohru Matsunaga'
'Richard G. E. Pinch''Carl-Arndt Krapp'

'Maria Aretoulaki''Laci Jalics'
'Jue Wu'

'Astrid Kiehn''Jörg Schwerdt'

'Hoon Chang'
'P. N. Denbigh'
'Tae-Hyeon Kim''Mark L. G. Althouse''Akira Hasegawa'


'Brynjulf Owren'
'Chris McConnell'
'Maic Masuch'
'Jonathan K. Riek''Petri Mähönen'

'Josep Lluís

'David Scott Warren'
'Cai Min'
'Ashraf A. Kassim''Yeol Je Cho'

'Ji-Chien Lee''Michael G. Kazantzakis''Arnaud Jacquin'


'Michael Healy''Denis Lalanne'

'Etienne Craye'
'Hoang-Lan Nguyen Thi''Rachel Cardell-Oliver''Laurie S. Keller'
'Yong-Fong Lee''Kamil Ugurbil'


'Jungsoon P. Yoo''Lawrence H. Ozarow'

'Sergei Starchenko'
'Madge Gleeson'

'Gerald T. Candela'
'Jonas Kuhn'
'Ken Nakaoka'
'Joseph Yan'
'K. R. Pai'
'Thomas Schimming'
'Carol Wood''Krishna Balachandran'
'John N. Coleman''Ram D. Gopal'

'Robert B. Jones'
'Vincent Schmitt'

'Khalid Rahmat''Kai R. T. Larsen'

'Michael Sfyrakis''Ioannis Haritantis'

'Youngjik Lee'
'Steve Y. Chiu'
'Gerd Szwillus'
'Moshe Shaked'
'Inderjeet Mani'
'S. Jonathan Chapman'
'Peter Sterk'
'Julia Abrahams'
'Ole Hansen''Kenneth L. Melmon'
'Oliver T. W. Yu'
'Slimane Ben Slimane'

'Aline de P. Nascimento''Guangzhou Zeng''T. Es-sqalli'


'Sriram V. Pemmaraju''Joseph H. Nurre'
'Isambo Karali'
'Gal A. Kaminka''Kai-Yeung Siu'

'Lei Sheng'
'Lawrence Chung'

'Jean-F

...

'Sunil Gupta''Michel Bonjour'
'Cheeha Kim'

'Welf Löwe'"Marcelo Sant'Anna"

'Nikolay A. Moldovyan'
'Walter A. Sedelow Jr.'
'Jaydeep Balakrishnan'
'Peter J. Harley'
'Symeon Retalis''Cheng-Hua Wang'

'Rose P. Tsang''Wen-Jong Fang'

'John Sapsford-Francis''Kunihiro Fujiyoshi'
'Tony Przygienda'

'Suhail S. Saquib'
'Masahiro Hori'
'Foo-Tim Chau'
'Janet Wiles'
'David A. Wroblewski'
'Heinz Dibold''Jeffery L. Kennington''Jörg Kluge''Douglas Chubb'


'Chun-Chin Fang'

'Xin-Zhi Zheng''Aida G. Romashchenko'
'Sreenivasa A. Raghavan'
"Pierre L'Ecuyer"
'Zhaohui Huang'

'Robert Schweikert''Vara Varavithya'
'Fereydoun Maali'
'Haiyun Yang'

'Yann Le Guyadec'
'Leen Breure'
'Brian H. Mayoh'
'Norbert Hegyvári''Eleni Stroulia'
'Christopher C. Boucher'
'Salvatore Lo Nigro''Mohand-Said Hacid'
'Glenn Jennings'


'John A. Bate''Christopher Sauer''Marijn van Eupen'

'Michael Clifton''Christine Urquhart'

'Jürgen Herzberger'
'Volker Gruhn''Venkateswarlu Kolluri'
'Thilo-Thomas Frieß'


'Fabio M. Chiussi''Stephen 

'Yoshinori Ogata''Hans-Peter Frei'

'Tracy Camp''Gerald T. Candela'

'Douglas A. Abraham''Barry Merriman'

'Hitoshi Naruse''James Kilbury'

'Vinti Doshi''John B. Moore''Ian S. Curthoys'


'Christian Lemmen''Stephan Mehl'

'Fritz Boschen'
'Nello Scarabottolo''Yung Ting'
'Chung-Shyan Liu'

'Ee Ping Ong'
'Derek L. Beatty''P. Jourlin''Vítor Santos'

'Joseph Fong''Junho Shim''Angelien Sanderman'


'Daniel Charlebois''Clarence N. W. Tan'


'Sudipta Mahapatra'
'Jim Lambert''Joseph M. Maubach'

'Jürgen Eckerle'
'Nicolas Merlet'
'Joan Vila i Carbó'
'John R. Clymer''Matthias Hillebrand''Hannu Salonen''Eric D. Glendening'

'Urs-Viktor Marti'


'Douglas Miles'
'Tal Grossman''Marcelo G. Rubinstein'

'Oliver Costich''Olivier Guibert''Aaron J. Goldberg'

'Charles E. Knadler Jr.'

'Irina Kononenko'
'Takeshi Chusho'
'Ernest Foo'
'Mark A. Shifman'
'Stephen M. Scalera''Peter G. Thomas'
'Gerhard Steinke'
'Silvana Badaloni'

'Helmut Prodinger'
'Paul A. Rubin'
'Roberto Revetria''Plamen Y. Yalamov'

'Robert 

'Paulo Nazareno Maia Sampaio''Joseph R. Litko'

'Megumi Kameyama''Jan Linden'
'Wilhelm Held'

'C. Mary Shapcott''Dino Farinacci'

'Eugene J. H. Kerckhoffs'
'Satoshi Yoshiara''Christian Prehofer'
'Robert Kennedy'

'Jean Pierre Courtial''Henk Alblas'
'W. Harvey Holmes'

'Lee Markosian'
'N. K. Bose''Jon Lee 0001'

'Gerald Pfeifer''F. Meyer'

"E. A. O'Brien"
'Chye Hwang Yan'
'Reiner Wilhelms-Tricarico'
'Frederic Duvivier''Federico Silla''Tsann-Shyong Liu'

'Samuel Sideman'
'Kazunori Itoh''Raoul Tawel'

'Nick K. Treadgold'

'Ahmed A. Rafea''Gene T. J. Wuu''Wie Ming Lim''K. Brown'

'Mark J. T. Smith'


'Asuman Sünbül''Ying Hao'

'Patrick Saint-Dizier'
'Lena Bonsiepen'
'Kuansan Wang''Il Y. Kim'

'Qiming Zhou''James Ze Wang''Nicole Beringer'

'Nitin H. Vaidya''Karen C. Davis'


'Maria João Alves'
'Peter M. Hastings'
'Norio Nakagawa''Brian Kantor'

'Jae-Beom Lee'"Brian O'Neill"'Lei Zhu'

'Mark Christon'

'Klaus Feldmann'
'Klaus Lux'
'Uwe Jäger'
'Stathes Hadjiefthymiades'
'Robert F. Harrison'
'J

'Ravi A. Dirckze''Chris Jones 0001'

'Alejandro P. Buchmann'
'Martin David'
'Michael K. Birbas'
'Patrick Doherty'
'Dimitris K. Agrafiotis'
'Hee-Seon Park'
'Fernando Ferri''Jong Won Yi'

'Susanne A. Huber'
'Tetsushi Oka'
'David T. Borup'
'Jesús D. García-Consuegra'
'Hongyao Zhu''Izzet Sirin'

'Tse-Hsin Liu'
'Mamoru Hosaka''James T. Klosowski'
'Petra Schumann''Dmitri Boulanger'

'Jingsong Liu'

'Ahmad S. Tolba''Moheb R. Girgis'

'Pablo A. Parrilo''Javier Abad'

'Pedro Luis Dias Peres''Hongzhou Tan'
'Philip Newcomb'

'Kevin J. Kerns''Henning Schmidt'

'Ingrid Y. Bucher''R. Gopalakrishnan'

'Dan O. Popa''John Tsimbinos'

'David Gray''MaryEllen C. Sievert''Chiara Scapoli'


'Joseph M. Winograd''Sergei M. Obraztsov'
'Janne Takala'

'Takeshi Shinohara'
'C. K. Un'
'Luigi Verri''Charles E. Kahn Jr.'

'Alberto Torres''Donna Crystal Llewellyn'
'Eduardo Costa Pereira'

'Patrice Brémond-Grégoire''Frank Duncalfe'
'Milan Randic'

'Patrice Ossona de Mendez''Gerhard Werling'
'Jing Yuan''Marc Weeber'




'Jousuke Kuroiwa''Joong-Goo Song'

'Cesar Guerra-Salcedo'
'Eliahu Ibrahim Jury''Steven Fincke'

'Dirk Menke''L. Reis'

'J. Alex Stark'
'Wil Dekkers''Mary L. Comer''Debabrata Saha'

'Bruce F. Naylor'

'Jizhou Sun''Robert M. Hayes'

'Patrick D. McDaniel'
'Yu T. Su'
'Raj K. Singh'
'Paul R. Taylor'
"Peter J. O'Shea"
'Ladislav Luksan'
'Ronald K. Pearson'
'Michael Morris'
'Birgit Burmeister'
'Michel Soto'
'Henry Baltes'
'Jeffrey S. Beis''Kenzo Nonami'

'Hua Hua'
'Ernest Barany''Tim Zajic'

'Emma Wittenmark''Jean-Gabriel Attali''Paola Inverardi'

'Keitha A. Murray'

'L. Chan'
'Pekka Appelqvist'
'John A. Reisner''Jeffrey J. Derby'

'Clive Warren''Monica L. Visinsky'
'Michael Holdgaard'

'Nancy Roderer''Valerie Florance'

'William C. Sasso'
'Ephraim Zehavi'
'Lei F. Willems'
'Anthony J. Lapadula'
'James J. Buckley'
'Walter J. Trybula'
'Ronald Waxman'
'Michael Haag'
'Peter Santago'
'Mark S. Fox'
'Maneesh Agrawala'
'Palle M. Pedersen'
'Gene Tsudik'
'Pieter Unema'
'Larry M. Manevitz''Chong-Yung Chi

In [101]:
shortest_paths = p.map(extract_lenght_short_path,sample[['source','target']].values)

In [102]:
len(shortest_paths)

38103

In [103]:
sample['lenght_short_path'] = shortest_paths

In [104]:
sample['lenght_short_path'].value_counts()

1.0     18103
7.0      1252
6.0      1015
8.0       978
9.0       630
5.0       482
10.0      258
11.0      121
4.0       113
12.0       45
3.0        28
13.0       18
2.0         6
15.0        4
14.0        3
Name: lenght_short_path, dtype: int64

In [105]:
#sample[['source','target']].tail().values

In [106]:
#sample[['source','target']].tail().apply(lambda x: extract_lenght_short_path(x[0],x[1]), axis=1)

In [107]:
#sample['lenght_short_path'] = sample[['source','target']].apply(lambda x: extract_lenght_short_path(x[0],x[1]), axis=1)

In [108]:
sample[sample.sum_of_neighbors==4].iloc[0].target

'Mineo Kaneko'

In [109]:
sample[sample.sum_of_neighbors==4].iloc[0].source

'Satoshi Tayu'

### Clustering index

In [110]:
def clustering_index(node_a,node_b):
    d = nx.clustering(G, nodes=[node_a,node_b])
    return sum(d.values())
    
    

In [111]:
sample['clustering_index_sum'] = sample[['source','target']].apply(lambda x: clustering_index(x[0],x[1]), axis=1)

In [112]:
import datetime

In [113]:
today = datetime.datetime.today()

In [114]:
today.strftime('%Y-%m-%d')

'2021-01-06'

In [115]:
sample.to_csv('sample_features_test'+today.strftime('%Y-%m-%d')+'.csv')

In [116]:
sample.head()

,source,target,connected,sum_of_papers,sum_of_neighbors,log_secundary_neighbors,lenght_short_path,clustering_index_sum
0,Hideaki Takanobu,Kayoko Ohtsuki,1.0,17,34,10.757477,1.0,1.407389
1,Joseph Douglas Horton,Alejandro López-Ortiz,1.0,26,25,11.075040,1.0,0.752381
2,Siddhartha R. Dalal,Ashish Jain,1.0,19,11,7.494430,1.0,0.800000
3,Laurence Melloul,Armando Fox,1.0,21,16,8.034955,1.0,1.153846
4,Riccardo Bettati,Wei Zhao 0001,1.0,76,25,9.294498,1.0,1.109524
